# Analysis of the UN's World Happiness Index with machine learning

Maaike de Jong  
June 2020  
  
See the repository's [README](https://github.com/maaikedj/happiness-machine-learning/blob/master/README.md) file for background and details on the analysis and data.  

### Notebook 4: Data visualization with graphs and maps
In this notebook I visualize the relationship between the World Happiness Index with the World Bank's World Development Indicators.
  

In [ ]:
# import packages

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely import wkt

In [ ]:
# import main data file

df = pd.read_csv('dfML.csv')
df.head()

In [ ]:
# rename some of the columns (see previous notebook)

df2 = df.rename(columns = {'Score mean': 'Happiness score',
                            'CO2 emissions (metric tons per capita)': 'CO2 emission per capita',
                            'GDP per capita (current US$)': 'GDP per capita',
                            'Individuals using the Internet (% of population)': 'Access to electricity %',
                            'Land area (sq. km)': 'Land area',
                            'Life expectancy at birth, total (years)': 'Life expectancy',
                            'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)': 'Air pollution',
                            'People using at least basic drinking water services (% of population)': 'Drinking water services',
                            'Population density (people per sq. km of land area)': 'Population density',
                            'Population growth (annual %)': 'Population growth',
                            'Primary education, duration (years)': 'Primary education years',
                            'Proportion of seats held by women in national parliaments (%)': 'Women in parliament %',
                            'Renewable energy consumption (% of total final energy consumption)': 'Renewable energy %',
                            'Secondary education, duration (years)': 'Secondary education years',
                            'Terrestrial protected areas (% of total land area)': 'Protected land %',
                            'Urban population (% of total population)': 'Urban population',
                            'Refugees country of origin (% of total population': 'Refugees %'})

df2.head()

In [ ]:
# Add columns with log versions of some variables for plotting



In [ ]:
df2['CO2 emission per capita (log)'] = np.log10(df2['CO2 emission per capita'])
df2['GDP per capita (log)'] = np.log10(df2['GDP per capita'])
df2['Land area (log)'] = np.log10(df2['Land area'])
df2['Air pollution (log)'] = np.log10(df2['Air pollution'])
df2['Population density (log)'] = np.log10(df2['Population density'])
df2['Refugees % (log)'] = np.log10(df2['Refugees %'])

In [ ]:
df2.head()

In [ ]:
# Make few more figures to show the correlation between happiness score and top three variables
# Top three variables are: GDP, life expectancy and refugees

sns.set()
sns.set_style('white')
sns.set_color_codes('pastel')

f, ax = plt.subplots(figsize=(12, 9))

sns.regplot(x = 'Happiness score', y = 'GDP per capita (log)', color = 'royalblue', data = df2)

ax.tick_params(axis='both', which='major', labelsize=20) 
ax.tick_params(axis='both', which='minor', labelsize=20)
ax.set_xticks(range(3, 9, 1))
#ax.set_yticks(range(0, 7, 1))

# ax.legend(loc="lower right", frameon=True, fontsize = 20)

plt.xlabel('Happiness score', fontsize=24)
plt.ylabel('GDP per capita (log)', fontsize=24)
plt.suptitle('Regression between Happiness score and GDP per capita', fontsize=24)

sns.despine()

In [ ]:
sns.set()
sns.set_style('white')
sns.set_color_codes('pastel')

f, ax = plt.subplots(figsize=(12, 9))

sns.regplot(x = 'Happiness score', y = 'Life expectancy', color = 'mediumseagreen', data = df2)

ax.tick_params(axis='both', which='major', labelsize=20) 
ax.tick_params(axis='both', which='minor', labelsize=20)
ax.set_xticks(range(3, 9, 1))
#ax.set_yticks(range(0, 7, 1))

# ax.legend(loc="lower right", frameon=True, fontsize = 20)

plt.xlabel('Happiness score', fontsize=24)
plt.ylabel('Life expectancy', fontsize=24)
plt.suptitle('Regression between Happiness score and Life expectancy', fontsize=24)

sns.despine()

In [ ]:
sns.set()
sns.set_style('white')
sns.set_color_codes('pastel')

f, ax = plt.subplots(figsize=(12, 9))

sns.regplot(x = 'Happiness score', y = 'Refugees % (log)', color = 'coral', data = df2)

ax.tick_params(axis='both', which='major', labelsize=20) 
ax.tick_params(axis='both', which='minor', labelsize=20)
ax.set_xticks(range(3, 9, 1))
#ax.set_yticks(range(0, 7, 1))

# ax.legend(loc="lower right", frameon=True, fontsize = 20)

plt.xlabel('Happiness score', fontsize=24)
plt.ylabel('Refugees % (log)', fontsize=24)
plt.suptitle('Regression between Happiness score and percentage of refugees of the population', fontsize=24)

sns.despine()

In [ ]:
# import world map geodataframe. The lowres worldmap dataset from Natural Earth can be imported directly from geopandas

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.rename(columns = {'name': 'Country'}, inplace = True)
world.head()

In [ ]:
world.shape

In [ ]:
df2.shape

In [ ]:
# Will merge this with main df on country. First check whether country names are the samelist15 = hap15_s1['Country'].values.tolist()
list1 = df2['Country'].values.tolist()

list2 = world['Country'].values.tolist()

In [ ]:
sorted(list(set(list1) - set(list2)))

In [ ]:
sorted(list(set(list2) - set(list1)))

In [ ]:
# replace names

world2 = world.replace({'Country': {'Bosnia and Herz.': 'Bosnia and Herzegovina', 'Central African Rep.': 'Central African Republic', 'Congo': 'Congo (Brazzaville)', 'Dem. Rep. Congo': 'Congo (Kinshasa)', 'Czechia': 'Czech Republic', 'Dominican Rep.': 'Dominican Republic', "Côte d'Ivoire": 'Ivory Coast', 'Palestine': 'Palestinian Territories', 'S. Sudan': 'South Sudan', 'eSwatini': 'Swaziland', 'United States of America': 'United States'}})


In [ ]:
# create new world df without Antarctica: 

world3 = world2[world2['Country'] != 'Antarctica']

In [ ]:
# join this df with the main df

dfgeo = pd.merge(world3, df2, on = 'Country', how = 'left')

In [ ]:
gdf = gpd.GeoDataFrame(dfgeo, geometry = 'geometry')
gdf.head()

In [ ]:
type(gdf['geometry'])

In [ ]:
# Create choropleth maps of happiness score and other variables per country

# first check whether the country map data is complete
plt.rcParams['figure.figsize'] = [16,8]
gdf.plot(facecolor = 'silver', edgecolor = 'grey');

In [ ]:
# Fill happiness score NaNs with zeroes
gdf['Happiness score'] = gdf['Happiness score'].fillna(value = 0)

In [ ]:
# plot the map with happiness scores

plt.rcParams['figure.figsize'] = [20, 10]
fig, ax = plt.subplots(1, 1)

gdf.plot(column='Happiness score', legend=True, ax = ax, cmap='summer_r', scheme='user_defined', classification_kwds = {'bins':[3, 4, 5, 6, 7, 8]})

cmap = cm.get_cmap('summer_r')
patch1 = mpatches.Patch(color=cmap(0.0), label = 'No data')
patch2 = mpatches.Patch(color=cmap(0.2), label = '3 - 4')
patch3 = mpatches.Patch(color=cmap(0.4), label = '4 - 5')
patch4 = mpatches.Patch(color=cmap(0.6), label = '5 - 6')
patch5 = mpatches.Patch(color=cmap(0.8), label = '6 - 7')
patch6 = mpatches.Patch(color=cmap(1.0), label = '7 - 8')

plt.legend(handles = [patch1, patch2, patch3, patch4, patch5, patch6], prop = {'size':16}, loc = 'lower left', frameon = False)

plt.show()


In [ ]:
ax.set_axis_off()
ax.get_figure()

In [ ]:
# investigate GDP per capita

gdf['GDP per capita'].describe()


In [ ]:
# Fill GDP NaNs with zeroes
gdf['GDP per capita'] = gdf['GDP per capita'].fillna(value = 0)

In [ ]:
# plot map with GDP

plt.rcParams['figure.figsize'] = [20, 10]
fig, ax = plt.subplots(1, 1)

gdf.plot(column='GDP per capita', legend=True, ax = ax, cmap='summer_r', scheme='user_defined', classification_kwds = {'bins':[100, 1000, 5000, 10000, 50000, 200000]})

cmap = cm.get_cmap('summer_r')
patch1 = mpatches.Patch(color=cmap(0.0), label = 'No data')
patch2 = mpatches.Patch(color=cmap(0.2), label = '100 - 1k')
patch3 = mpatches.Patch(color=cmap(0.4), label = '1k - 5k')
patch4 = mpatches.Patch(color=cmap(0.6), label = '5k - 10k')
patch5 = mpatches.Patch(color=cmap(0.8), label = '10k - 50k')
patch6 = mpatches.Patch(color=cmap(1.0), label = '50k - 100k')

plt.legend(handles = [patch1, patch2, patch3, patch4, patch5, patch6], prop = {'size':16}, loc = 'lower left', frameon = False)

plt.show()

In [ ]:
ax.set_axis_off()
ax.get_figure()